In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import string
import os
import requests

from requests import get
from bs4 import BeautifulSoup
from warnings import warn
from time import sleep
from random import randint
from xgboost import XGBClassifier
from tqdm import tqdm
from time import sleep

from pathlib import Path
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split, GridSearchCV
from nltk.stem.porter import *
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from tensorflow.keras import layers, Sequential, callbacks, optimizers, utils, models, applications
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

2023-08-12 09:51:58.902299: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-12 09:51:59.123421: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-12 09:51:59.123448: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-08-12 09:51:59.167356: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-12 09:52:00.451228: W tensorflow/stream_executor/platform/de

In [2]:
# folder_path = '/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/csv/'
# dataframes = []
# for file_name in os.listdir(folder_path):
#     if file_name.endswith('.csv'):
#         file_path = os.path.join(folder_path, file_name)
#         df = pd.read_csv(file_path)
#         dataframes.append(df)
# merged_df = pd.concat(dataframes, ignore_index=True)

In [3]:
merged_df = pd.read_csv("/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/long_sypnosis.csv")

merged_df.shape

(4624, 6)

In [4]:
merged_df.head(5)

,Unnamed: 0,movie,imdb_id,genre,image_url,sypnosis
0,0,Ramayana: The Legend of Prince Rama,tt0259534,"['Animation', ' Action', ' Adventure']",http://img.omdbapi.com/?i=tt0259534&h=600&apik...,An anime adaptation of the Hindu epic the Rama...
1,1,The Dark Knight,tt0468569,"['Action', ' Crime', ' Drama']",http://img.omdbapi.com/?i=tt0468569&h=600&apik...,When the menace known as the Joker wreaks havo...
2,2,The Lord of the Rings: The Return of the King,tt0167260,"['Action', ' Adventure', ' Drama']",http://img.omdbapi.com/?i=tt0167260&h=600&apik...,Gandalf and Aragorn lead the World of Men agai...
3,3,Spider-Man: Across the Spider-Verse,tt9362722,"['Animation', ' Action', ' Adventure']",http://img.omdbapi.com/?i=tt9362722&h=600&apik...,"Miles Morales catapults across the Multiverse,..."
4,4,The Message,tt0075143,"['Action', ' Adventure', ' Biography']",http://img.omdbapi.com/?i=tt0075143&h=600&apik...,"The story of prophet ""Muhammad"" and the delive..."


In [5]:
merged_df = merged_df.drop(columns=['Unnamed: 0'])

In [6]:
merged_df = merged_df.drop_duplicates()
merged_df.shape

(4624, 5)

In [7]:
merged_df.head(5)

,movie,imdb_id,genre,image_url,sypnosis
0,Ramayana: The Legend of Prince Rama,tt0259534,"['Animation', ' Action', ' Adventure']",http://img.omdbapi.com/?i=tt0259534&h=600&apik...,An anime adaptation of the Hindu epic the Rama...
1,The Dark Knight,tt0468569,"['Action', ' Crime', ' Drama']",http://img.omdbapi.com/?i=tt0468569&h=600&apik...,When the menace known as the Joker wreaks havo...
2,The Lord of the Rings: The Return of the King,tt0167260,"['Action', ' Adventure', ' Drama']",http://img.omdbapi.com/?i=tt0167260&h=600&apik...,Gandalf and Aragorn lead the World of Men agai...
3,Spider-Man: Across the Spider-Verse,tt9362722,"['Animation', ' Action', ' Adventure']",http://img.omdbapi.com/?i=tt9362722&h=600&apik...,"Miles Morales catapults across the Multiverse,..."
4,The Message,tt0075143,"['Action', ' Adventure', ' Biography']",http://img.omdbapi.com/?i=tt0075143&h=600&apik...,"The story of prophet ""Muhammad"" and the delive..."


In [8]:
merged_df["sypnosis"].notna().sum()

4624

In [9]:
merged_df

,movie,imdb_id,genre,image_url,sypnosis
0,Ramayana: The Legend of Prince Rama,tt0259534,"['Animation', ' Action', ' Adventure']",http://img.omdbapi.com/?i=tt0259534&h=600&apik...,An anime adaptation of the Hindu epic the Rama...
1,The Dark Knight,tt0468569,"['Action', ' Crime', ' Drama']",http://img.omdbapi.com/?i=tt0468569&h=600&apik...,When the menace known as the Joker wreaks havo...
2,The Lord of the Rings: The Return of the King,tt0167260,"['Action', ' Adventure', ' Drama']",http://img.omdbapi.com/?i=tt0167260&h=600&apik...,Gandalf and Aragorn lead the World of Men agai...
3,Spider-Man: Across the Spider-Verse,tt9362722,"['Animation', ' Action', ' Adventure']",http://img.omdbapi.com/?i=tt9362722&h=600&apik...,"Miles Morales catapults across the Multiverse,..."
4,The Message,tt0075143,"['Action', ' Adventure', ' Biography']",http://img.omdbapi.com/?i=tt0075143&h=600&apik...,"The story of prophet ""Muhammad"" and the delive..."
...,...,...,...,...,...
4619,Bad Girls,tt0109198,"['Action', ' Adventure', ' Drama']",http://img.omdbapi.com/?i=tt0109198&h=600&apik...,Four prostitutes join together to travel the O...
4620,Renegade,tt0276830,"['Adventure', ' Fantasy', ' Western']",http://img.omdbapi.com/?i=tt0276830&h=600&apik...,"In 1870, after a brutal run-in with an outlaw ..."
4621,Wild Wild West,tt0120891,"['Action', ' Comedy', ' Sci-Fi']",http://img.omdbapi.com/?i=tt0120891&h=600&apik...,The two best special agents in the Wild West m...
4622,The Ridiculous 6,tt2479478,"['Action', ' Adventure', ' Comedy']",http://img.omdbapi.com/?i=tt2479478&h=600&apik...,An outlaw who was raised by Native Americans d...


In [10]:
merged_df[merged_df["genre"] == "Drama"]

,movie,imdb_id,genre,image_url,sypnosis


In [11]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer

In [12]:
def cleaning(sentence):
    if isinstance(sentence, str):
    # Basic cleaning
        sentence = sentence.strip() ## remove whitespaces
        sentence = sentence.lower() ## lowercase 
        sentence = ''.join(char for char in sentence if not char.isdigit()) ## remove numbers

        # Advanced cleaning
        for punctuation in string.punctuation:
            sentence = sentence.replace(punctuation, '') ## remove punctuation

        tokenized_sentence = word_tokenize(sentence) ## tokenize 
        stop_words = set(stopwords.words('english')) ## define stopwords

        tokenized_sentence_cleaned = [ ## remove stopwords
            w for w in tokenized_sentence if not w in stop_words
        ]

        lemmatized = [
            WordNetLemmatizer().lemmatize(word, pos = "v") 
            for word in tokenized_sentence_cleaned
        ]

        cleaned_sentence = ' '.join(word for word in lemmatized)

        return cleaned_sentence

In [13]:
# merged_df["clean_plot"] = merged_df["plot"].apply(cleaning)

In [14]:
merged_df["clean_sypnosis"] = merged_df["sypnosis"].apply(cleaning)

In [15]:
merged_df["clean_title"] = merged_df["movie"].apply(cleaning)

In [16]:
merged_df.isna().sum()

movie             0
imdb_id           0
genre             0
image_url         0
sypnosis          0
clean_sypnosis    0
clean_title       0
dtype: int64

In [17]:
merged_df["genre"] = (
    merged_df["genre"]
    .apply(eval)
    .apply(lambda x: [genre.strip() for genre in x])
    )

In [18]:
merged_df.shape

(4624, 7)

In [19]:
merged_df = merged_df.dropna()

In [20]:
merged_df.shape

(4624, 7)

In [21]:
#check the genre we have
merged_df["genre"].explode().value_counts()

Drama        2824
Comedy       1495
Adventure    1045
Action        994
Crime         806
Romance       647
Biography     591
Thriller      544
Horror        511
Mystery       504
Animation     405
Fantasy       387
Sci-Fi        360
Family        312
History       298
Music         261
War           201
Sport         187
Musical       106
Western       104
Film-Noir      59
Name: genre, dtype: int64

In [22]:
merged_df.describe()

,movie,imdb_id,genre,image_url,sypnosis,clean_sypnosis,clean_title
count,4624,4624,4624,4624,4624,4624,4624
unique,4515,4624,383,4624,4624,4624,4324
top,A Star Is Born,tt0259534,"[Animation, Adventure, Comedy]",http://img.omdbapi.com/?i=tt0259534&h=600&apik...,An anime adaptation of the Hindu epic the Rama...,anime adaptation hindu epic ramayana lord ram ...,
freq,4,1,223,1,1,1,36


In [23]:
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(merged_df["genre"])

# transform target variable
y = multilabel_binarizer.transform(merged_df['genre'])

In [24]:
genre_names = multilabel_binarizer.classes_

# Adding 
for i in range(len(genre_names)):
    merged_df[f"{genre_names[i]}"] = y[:,i]

merged_df.shape

(4624, 28)

In [25]:
merged_df.head(5)

,movie,imdb_id,genre,image_url,sypnosis,clean_sypnosis,clean_title,Action,Adventure,Animation,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,Ramayana: The Legend of Prince Rama,tt0259534,"[Animation, Action, Adventure]",http://img.omdbapi.com/?i=tt0259534&h=600&apik...,An anime adaptation of the Hindu epic the Rama...,anime adaptation hindu epic ramayana lord ram ...,ramayana legend prince rama,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,The Dark Knight,tt0468569,"[Action, Crime, Drama]",http://img.omdbapi.com/?i=tt0468569&h=600&apik...,When the menace known as the Joker wreaks havo...,menace know joker wreak havoc chaos people got...,dark knight,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Lord of the Rings: The Return of the King,tt0167260,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0167260&h=600&apik...,Gandalf and Aragorn lead the World of Men agai...,gandalf aragorn lead world men saurons army dr...,lord ring return king,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Spider-Man: Across the Spider-Verse,tt9362722,"[Animation, Action, Adventure]",http://img.omdbapi.com/?i=tt9362722&h=600&apik...,"Miles Morales catapults across the Multiverse,...",miles morales catapult across multiverse encou...,spiderman across spiderverse,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,The Message,tt0075143,"[Action, Adventure, Biography]",http://img.omdbapi.com/?i=tt0075143&h=600&apik...,"The story of prophet ""Muhammad"" and the delive...",story prophet muhammad delivery message god allah,message,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
# from imblearn.under_sampling import RandomUnderSampler
# from sklearn.datasets import make_classification
# from collections import Counter

# # Generate a sample imbalanced dataset
# X, y = make_classification(n_classes=2, class_sep=2, weights=[0.1, 0.9], n_informative=3, n_redundant=1, flip_y=0, n_features=20, n_clusters_per_class=1, n_samples=1000, random_state=10)

# # Count the class distribution before undersampling
# print("Class distribution before undersampling:", Counter(y))

# # Apply random undersampling
# rus = RandomUnderSampler(random_state=42)
# X_resampled, y_resampled = rus.fit_resample(X, y)

# # Count the class distribution after undersampling
# print("Class distribution after undersampling:", Counter(y_resampled))

In [27]:
# X = merged_df[["clean_sypnosis","clean_title"]]
# y = merged_df.drop(columns=["movie","imdb_id","genre","image_url","sypnosis","clean_sypnosis","clean_title"])

In [28]:
condition1 =  (merged_df['Music'] == 1)  | (merged_df['Biography'] == 1) | (merged_df['Sport'] == 1) |(merged_df['Musical'] == 1) |(merged_df['War'] == 1)|(merged_df['Western'] == 1)|(merged_df['History'] == 1)|(merged_df['Film-Noir'] == 1)|(merged_df['Family'] == 1)

rows_to_drop1 = merged_df[condition1].index
test_df = merged_df.drop(rows_to_drop1)

test_df["genre"].explode().value_counts()

Drama        1485
Comedy        980
Adventure     821
Action        796
Crime         613
Horror        493
Thriller      481
Mystery       477
Romance       419
Animation     364
Sci-Fi        351
Fantasy       296
Name: genre, dtype: int64

In [29]:
condition2 = (test_df['Drama'] == 1)  & (test_df['Romance'] != 1)& (test_df['Thriller'] != 1)& (test_df['Biography'] != 1)& (test_df['Horror'] != 1)& (test_df['Mystery'] != 1)& (test_df['Animation'] != 1)& (test_df['Sci-Fi'] != 1)& (test_df['Fantasy'] != 1)& (test_df['History'] != 1)& (test_df['Family'] != 1)& (test_df['Music'] != 1)
rows_to_drop2 = test_df[condition2].index
test_df2 = test_df.drop(rows_to_drop2)

   
test_df2["genre"].explode().value_counts()

Drama        1000
Comedy        801
Adventure     693
Action        620
Horror        493
Thriller      481
Mystery       477
Romance       419
Crime         415
Animation     364
Sci-Fi        351
Fantasy       296
Name: genre, dtype: int64

In [30]:
condition3 = (test_df2['Comedy'] == 1)  & (test_df2['Drama'] == 1)
# & (test_df['Romance'] != 1)& (test_df['Thriller'] != 1)& (test_df['Biography'] != 1)& (test_df['Horror'] != 1)& (test_df['Mystery'] != 1)& (test_df['Animation'] != 1)& (test_df['Sci-Fi'] != 1)& (test_df['Fantasy'] != 1)& (test_df['History'] != 1)& (test_df['Family'] != 1)& (test_df['Music'] != 1)
rows_to_drop3 = test_df2[condition3].index
test_df3= test_df2.drop(rows_to_drop3)

   
test_df3["genre"].explode().value_counts()

Drama        821
Adventure    693
Comedy       622
Action       620
Horror       487
Thriller     480
Mystery      470
Crime        415
Animation    357
Sci-Fi       345
Romance      313
Fantasy      250
Name: genre, dtype: int64

In [31]:
# condition = (merged_df['Drama'] == 1) & (merged_df['Comedy'] == 1)
# rows_to_drop = merged_df[condition].head(500).index
# balancing_df = merged_df.drop(rows_to_drop)
# balancing_df.shape
# balancing_df["genre"].explode().value_counts()

In [32]:
# condition1 = (merged_df['Drama'] == 1)&(merged_df['Romance'] != 1)& (merged_df['Sci-Fi'] != 1)& (merged_df['Fantasy'] != 1)
# # & (merged_df['Crime'] != 1)& (merged_df['Romance'] != 1)& (merged_df['Thriller'] != 1)& (merged_df['Biography'] != 1)& (merged_df['Horror'] != 1)& (merged_df['Mystery'] != 1)& (merged_df['Animation'] != 1)& (merged_df['Sci-Fi'] != 1)& (merged_df['Fantasy'] != 1)& (merged_df['History'] != 1)& (merged_df['Family'] != 1)& (merged_df['Music'] != 1)
# # & (merged_df['War'] == 1)& (merged_df['Sport'] == 1)& (merged_df['Western'] == 1)& (merged_df['Musical'] == 1)& (merged_df['Film-Noir'] == 1)
# # & (merged_df['Adventure'] == 1)& (merged_df['Action'] != 1)
# # & (merged_df['Comedy'] != 1)

# rows_to_drop1 = merged_df[condition1].index
# # .head(1000).index
# test_df = merged_df.drop(rows_to_drop1)


   
# test_df["genre"].explode().value_counts()

In [33]:
# condition2 = (test_df['Comedy'] == 1)  & (test_df['Adventure'] == 1)
# # & (test_df['Romance'] != 1)& (test_df['Thriller'] != 1)& (test_df['Biography'] != 1)& (test_df['Horror'] != 1)& (test_df['Mystery'] != 1)& (test_df['Animation'] != 1)& (test_df['Sci-Fi'] != 1)& (test_df['Fantasy'] != 1)& (test_df['History'] != 1)& (test_df['Family'] != 1)& (test_df['Music'] != 1)
# rows_to_drop2 = test_df[condition2].index
# test_df2 = test_df.drop(rows_to_drop2)

   
# test_df2["genre"].explode().value_counts()

In [34]:
# condition3 = (test_df['Music'] == 1) | (test_df['Biography'] == 1) | (test_df['Sport'] == 1) |(test_df['Musical'] == 1) |(test_df['War'] == 1)|(test_df['Western'] == 1)|(test_df['History'] == 1)|(test_df['Film-Noir'] == 1)|(test_df['Family'] == 1)

# rows_to_drop3 = test_df[condition3].index
# test_df3 = test_df.drop(rows_to_drop3)

   
# test_df3["genre"].explode().value_counts()

In [35]:
# condition2 = (balancing_df['Drama'] == 1)
# rows_to_drop = balancing_df[condition2].head(500).index
# balancing_df2 = balancing_df.drop(rows_to_drop)
# balancing_df2["genre"].explode().value_counts()

In [36]:
test_df3.head(5)

,movie,imdb_id,genre,image_url,sypnosis,clean_sypnosis,clean_title,Action,Adventure,Animation,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,Ramayana: The Legend of Prince Rama,tt0259534,"[Animation, Action, Adventure]",http://img.omdbapi.com/?i=tt0259534&h=600&apik...,An anime adaptation of the Hindu epic the Rama...,anime adaptation hindu epic ramayana lord ram ...,ramayana legend prince rama,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,Spider-Man: Across the Spider-Verse,tt9362722,"[Animation, Action, Adventure]",http://img.omdbapi.com/?i=tt9362722&h=600&apik...,"Miles Morales catapults across the Multiverse,...",miles morales catapult across multiverse encou...,spiderman across spiderverse,1,1,1,...,0,0,0,0,0,0,0,0,0,0
5,Inception,tt1375666,"[Action, Adventure, Sci-Fi]",http://img.omdbapi.com/?i=tt1375666&h=600&apik...,A thief who steals corporate secrets through t...,thief steal corporate secrets use dreamsharing...,inception,1,1,0,...,0,0,0,0,0,1,0,0,0,0
8,Kill Bill: The Whole Bloody Affair,tt6019206,"[Action, Crime, Thriller]",http://img.omdbapi.com/?i=tt6019206&h=600&apik...,The Bride must kill her ex-boss and lover Bill...,bride must kill exboss lover bill betray wed r...,kill bill whole bloody affair,1,0,0,...,0,0,0,0,0,0,0,1,0,0
9,The Matrix,tt0133093,"[Action, Sci-Fi]",http://img.omdbapi.com/?i=tt0133093&h=600&apik...,When a beautiful stranger leads computer hacke...,beautiful stranger lead computer hacker neo fo...,matrix,1,0,0,...,0,0,0,0,0,1,0,0,0,0


## Split Data and Vectorize

In [37]:
X = test_df3[["clean_sypnosis","clean_title"]]
y = test_df3.drop(columns=["movie","imdb_id","genre","image_url","sypnosis","clean_sypnosis","clean_title"])

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [39]:
X_train.shape

(1504, 2)

In [40]:
X_train.head(5)

,clean_sypnosis,clean_title
4127,near future family reckon question love connec...,yang
3235,disease turn humanity live dead last man earth...,last man earth
2277,lovers marianne jeanpaul spend vacation villa ...,la piscine
2773,lonely doctor occupy unusual lakeside house be...,lake house
2713,composer sister discover reason able purchase ...,uninvited


In [41]:
y_train.shape 

(1504, 21)

In [42]:
y_train.head(5)

,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,Film-Noir,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
4127,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3235,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0
2277,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2773,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2713,0,0,0,0,0,0,0,0,1,0,...,1,0,0,1,0,0,0,0,0,0


In [43]:
# Instantiating the TfidfVectorizer
tf_idf_vectorizer = TfidfVectorizer(max_df=0.8, min_df=0.05, max_features=10000)
tf_idf_title_vectorizer = TfidfVectorizer(min_df=10, max_features=10000)
# tf_idf_vectorizer = TfidfVectorizer()

# Training it on the texts
X_train_plot_vec = pd.DataFrame(tf_idf_vectorizer.fit_transform(X_train["clean_sypnosis"]).toarray(),
                 columns = tf_idf_vectorizer.get_feature_names_out())
X_train_title_vec = pd.DataFrame(tf_idf_title_vectorizer.fit_transform(X_train["clean_title"]).toarray(),
                 columns = tf_idf_title_vectorizer.get_feature_names_out())
X_test_plot_vec =  pd.DataFrame(tf_idf_vectorizer.transform(X_test["clean_sypnosis"]).toarray(),
                 columns = tf_idf_vectorizer.get_feature_names_out())
X_test_title_vec =  pd.DataFrame(tf_idf_title_vectorizer.transform(X_test["clean_title"]).toarray(),
                 columns = tf_idf_title_vectorizer.get_feature_names_out())


In [44]:
X_test_title_vec

,age,batman,black,dark,day,dead,dragon,evil,ii,last,...,man,movie,night,one,part,rise,star,time,war,world
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
641,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
642,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
X_train_vec = pd.concat([X_train_plot_vec,X_train_title_vec], axis=1)

In [46]:
X_train_vec

,abandon,abduct,abilities,ability,able,aboard,abuse,accept,access,accident,...,man,movie,night,one,part,rise,star,time,war,world
0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.041754,0.0,0.0,0.0,0.031746,0.000000,0.0,0.000000,0.0,0.000000,...,0.642567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.073802,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.034398,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.028930,0.0,0.0,0.0,0.021995,0.000000,0.0,0.027088,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1499,0.000000,0.0,0.0,0.0,0.000000,0.052872,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1500,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1501,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1502,0.000000,0.0,0.0,0.0,0.000000,0.045063,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
X_train_vec.shape

(1504, 1673)

In [48]:
X_test_vec = pd.concat([X_test_plot_vec,X_test_title_vec], axis=1)

In [49]:
X_train_vec.head(5)

,abandon,abduct,abilities,ability,able,aboard,abuse,accept,access,accident,...,man,movie,night,one,part,rise,star,time,war,world
0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.041754,0.0,0.0,0.0,0.031746,0.0,0.0,0.000000,0.0,0.000000,...,0.642567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.073802,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.034398,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.028930,0.0,0.0,0.0,0.021995,0.0,0.0,0.027088,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
X_test_vec.shape

(645, 1673)

In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, accuracy_score

## OneVsRest logistic regression

### Simple OneVsRest logistic regression

In [52]:
log = LogisticRegression()
model_test_1 = OneVsRestClassifier(log)

model_test_1.fit(X_train_vec, y_train)

/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 3 is present in all training examples.
  warnings.warn(
/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 7 is present in all training examples.
  warnings.warn(
/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 9 is present in all training examples.
  warnings.warn(
/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 10 is present in all training examples.
  warnings.warn(
/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 12 is present in all training examples.
  warnings.warn(
/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages

OneVsRestClassifier(estimator=LogisticRegression())

In [53]:
y_pred = model_test_1.predict(X_test_vec)

In [54]:
accuracy_score(y_test, y_pred)

0.11162790697674418

In [55]:
f1_score(y_test, y_pred, average="micro")

0.5233500578927055

In [56]:
multilabel_binarizer.inverse_transform(y_pred)[1]

('Drama',)

In [57]:
test_df3[test_df3.clean_sypnosis == X_test.clean_sypnosis.iloc[1]]

,movie,imdb_id,genre,image_url,sypnosis,clean_sypnosis,clean_title,Action,Adventure,Animation,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
3972,The Cider House Rules,tt0124315,"[Drama, Romance]",http://img.omdbapi.com/?i=tt0124315&h=600&apik...,"A compassionate young man, raised in an orphan...",compassionate young man raise orphanage train ...,cider house rule,0,0,0,...,0,0,0,0,1,0,0,0,0,0


#### Grouping the topic

In [58]:
# from sklearn.decomposition import LatentDirichletAllocation

# # Instantiate the LDA 
# n_components = 7
# lda_model = LatentDirichletAllocation(n_components=n_components, max_iter = 500)

# # Fit the LDA on the vectorized documents
# lda_model.fit(X_train_vec)

In [59]:
# X_train_topic_mixture = lda_model.transform(X_train_vec)
# X_train_topic_df = pd.DataFrame(X_train_topic_mixture)
# X_train_topic_df

In [60]:
# X_test_topic_mixture = lda_model.transform(X_test_vec)
# X_test_topic_df = pd.DataFrame(X_test_topic_mixture)
# X_test_topic_df

### Training OneVsRest using topic grouping as input

In [61]:
model_test_2 = OneVsRestClassifier(log)
model_test_2.fit(X_train_topic_df, y_train)

NameError: name 'X_train_topic_df' is not defined

In [ ]:
y_pred_2 = model_test_2.predict(X_test_topic_df)
accuracy_score(y_test, y_pred_2)

In [ ]:
multilabel_binarizer.inverse_transform(y_pred_2)[0:10]

In [ ]:
multilabel_binarizer.inverse_transform(y_test)[0:10]

### Using both topic grouping as input + OneVsRest 

In [ ]:
# model_test_3 = OneVsRestClassifier(log)
# model_test_3.fit(X_train_topic_df, y_train)
# model_test_3.fit(X_train_vec, y_train)

In [62]:
# y_pred_3 = model_test_2.predict(X_test_topic_df)
# accuracy_score(y_test, y_pred_3)

## SGD

In [63]:
sgd = SGDClassifier(random_state = 42)
sgd_model = OneVsRestClassifier(sgd)
sgd_model.fit(X_train_vec,y_train)

/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 3 is present in all training examples.
  warnings.warn(
/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 7 is present in all training examples.
  warnings.warn(
/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 9 is present in all training examples.
  warnings.warn(
/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 10 is present in all training examples.
  warnings.warn(
/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 12 is present in all training examples.
  warnings.warn(
/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages

OneVsRestClassifier(estimator=SGDClassifier(random_state=42))

In [64]:
y_pred_sgd1 = sgd_model.predict(X_test_vec)

In [65]:
multilabel_binarizer.inverse_transform(y_pred_sgd1)[0:10]

[('Animation',),
 ('Drama', 'Romance', 'Thriller'),
 ('Drama', 'Mystery'),
 ('Action', 'Adventure'),
 ('Drama', 'Romance'),
 ('Drama', 'Sci-Fi'),
 ('Adventure', 'Drama', 'Thriller'),
 ('Adventure', 'Animation', 'Comedy'),
 (),
 ('Comedy',)]

In [66]:
accuracy_score(y_test, y_pred_sgd1)

0.14883720930232558

In [67]:
f1_score(y_test, y_pred_sgd1, average="micro")

0.6228044060732361

In [68]:
multilabel_binarizer.inverse_transform(y_pred_sgd1)[2]

('Drama', 'Mystery')

In [69]:
test_df3[test_df3.clean_sypnosis == X_test.clean_sypnosis.iloc[2]]

,movie,imdb_id,genre,image_url,sypnosis,clean_sypnosis,clean_title,Action,Adventure,Animation,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
3925,"Confess, Fletch",tt12718300,"[Comedy, Crime, Mystery]",http://img.omdbapi.com/?i=tt12718300&h=600&api...,After becoming the prime suspect in multiple m...,become prime suspect multiple murder fletch st...,confess fletch,0,0,0,...,0,0,0,1,0,0,0,0,0,0


## Random Forest Classifier

In [70]:
rfc = RandomForestClassifier(random_state = 42)
rfc_model = OneVsRestClassifier(rfc)

rfc_model.fit(X_train_vec,y_train)

/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 3 is present in all training examples.
  warnings.warn(
/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 7 is present in all training examples.
  warnings.warn(
/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 9 is present in all training examples.
  warnings.warn(
/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 10 is present in all training examples.
  warnings.warn(
/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 12 is present in all training examples.
  warnings.warn(
/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages

OneVsRestClassifier(estimator=RandomForestClassifier(random_state=42))

In [71]:
y_pred_rfc1 = rfc_model.predict(X_test_vec)

In [72]:
multilabel_binarizer.inverse_transform(y_pred_rfc1)[0:10]

[(),
 ('Drama', 'Romance'),
 (),
 ('Adventure',),
 ('Drama',),
 (),
 ('Drama',),
 ('Drama',),
 (),
 ()]

In [73]:
accuracy_score(y_test, y_pred_rfc1)

0.04496124031007752

In [74]:
f1_score(y_test, y_pred_rfc1, average="micro")

0.3923776526634907

In [75]:
condition = test_df3['clean_sypnosis'].isin(X_test['clean_sypnosis'][:10])
filtered_df = test_df3[condition]
filtered_df["genre"]

32                 [Action, Adventure]
317            [Action, Comedy, Crime]
678       [Adventure, Romance, Sci-Fi]
769      [Animation, Adventure, Drama]
1053    [Animation, Adventure, Comedy]
3415           [Action, Comedy, Crime]
3925          [Comedy, Crime, Mystery]
3950                  [Drama, Romance]
3972                  [Drama, Romance]
4016                  [Drama, Romance]
Name: genre, dtype: object

## Poly SVC

In [84]:
X_train1, X_val1, y_train1, y_val1 = train_test_split(merged_df['clean_sypnosis'], y, test_size = 0.3, random_state = 0)
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf1 = tfidf_vectorizer.fit_transform(X_train1)
X_val_tfidf1 = tfidf_vectorizer.transform(X_val1)
sgd = SGDClassifier()
lr = LogisticRegression(C=1.0)
svc = SVC(kernel='poly')
nb = MultinomialNB()
# One vs Restclassifier
model = OneVsRestClassifier(svc)
model.fit(X_train_tfidf1, y_train1)
y_pred1 = model.predict(X_val_tfidf1)
print(f'Accuracy score for SVC poly model is {accuracy_score(y_val1, y_pred1)}')
print(f'F1 score for SVC poly model is {f1_score(y_val1, y_pred1, average="micro")}')

ValueError: Found input variables with inconsistent numbers of samples: [4624, 2149]

In [80]:
sgd = SGDClassifier()
lr = LogisticRegression(C=1.0)
svc = SVC(kernel='poly')
nb = MultinomialNB()
# One vs Restclassifier
model = OneVsRestClassifier(svc)
model.fit(X_train_vec, y_train)
y_pred_sgd2 = model.predict(X_test_vec)
print(f'Accuracy score for SVC poly model is {accuracy_score(y_test, y_pred_sgd2)}')
print(f'F1 score for SVC poly model is {f1_score(y_test, y_pred_sgd2, average="micro")}')

/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 3 is present in all training examples.
  warnings.warn(
/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 7 is present in all training examples.
  warnings.warn(
/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 9 is present in all training examples.
  warnings.warn(
/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 10 is present in all training examples.
  warnings.warn(
/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 12 is present in all training examples.
  warnings.warn(
/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages

Accuracy score for SVC poly model is 0.027906976744186046
F1 score for SVC poly model is 0.1889055472263868


In [78]:
X_train1, X_val1, y_train1, y_val1 = train_test_split(merged_df['clean_sypnosis'], y, test_size = 0.3, random_state = 0)
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf1 = tfidf_vectorizer.fit_transform(X_train1)
X_val_tfidf1 = tfidf_vectorizer.transform(X_val1)
sgd = SGDClassifier()
lr = LogisticRegression(C=1.0)
svc = SVC(kernel='poly')
nb = MultinomialNB()
# One vs Restclassifier
model = OneVsRestClassifier(svc)
model.fit(X_train_tfidf1, y_train1)
y_pred1 = model.predict(X_val_tfidf1)
print(f'Accuracy score for SVC poly model is {accuracy_score(y_val1, y_pred1)}')
print(f'F1 score for SVC poly model is {f1_score(y_val1, y_pred1)}')

ValueError: Found input variables with inconsistent numbers of samples: [4624, 2149]

## Label Powerset

In [81]:
# using Label Powerset
from skmultilearn.problem_transform import LabelPowerset
# initialize label powerset multi-label classifier
classifier = LabelPowerset(LogisticRegression())
# train
classifier.fit(X_train_vec, y_train)
# predict
predictions_lp = classifier.predict(X_test_vec)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions_lp))
print("F1 = ",f1_score(y_test,predictions_lp, average="micro"))
print("\n")

/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


Accuracy =  0.24186046511627907
F1 =  0.5335187760778859




In [82]:
multilabel_binarizer.inverse_transform(predictions_lp)[0:10]

[('Adventure', 'Animation', 'Comedy'),
 ('Drama', 'Romance'),
 ('Crime', 'Drama', 'Mystery'),
 ('Action', 'Adventure', 'Sci-Fi'),
 ('Drama', 'Romance'),
 ('Crime', 'Drama', 'Thriller'),
 ('Drama', 'Romance'),
 ('Adventure', 'Animation', 'Comedy'),
 ('Adventure', 'Animation', 'Comedy'),
 ('Drama', 'Romance')]

In [83]:
condition = test_df3['clean_sypnosis'].isin(X_test['clean_sypnosis'][:10])
filtered_df = test_df3[condition]
filtered_df["genre"]

32                 [Action, Adventure]
317            [Action, Comedy, Crime]
678       [Adventure, Romance, Sci-Fi]
769      [Animation, Adventure, Drama]
1053    [Animation, Adventure, Comedy]
3415           [Action, Comedy, Crime]
3925          [Comedy, Crime, Mystery]
3950                  [Drama, Romance]
3972                  [Drama, Romance]
4016                  [Drama, Romance]
Name: genre, dtype: object

## Classifier Chains

In [ ]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain

# initialize classifier chains multi-label classifier
classifier = ClassifierChain(LogisticRegression())
# Training logistic regression model on train data
classifier.fit(X_train_vec, y_train)
# predict
predictions = classifier.predict(X_test_vec)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")

In [ ]:
multilabel_binarizer.inverse_transform(predictions)[0:3]

In [ ]:
condition = test_df3['clean_sypnosis'].isin(X_test['clean_sypnosis'][:3])
filtered_df = test_df3[condition]
filtered_df["genre"]